# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [1]:
from collections import defaultdict
from tqdm import tqdm

import pandas as pd
import random

In [2]:
bigram_data = pd.read_csv("bigram_data.txt", sep="\t", header=None, on_bad_lines="skip")
bigram_data = bigram_data.dropna()
bigram_data

,0,1,2
0,275,a,a
1,31,a,aaa
2,29,a,all
3,45,a,an
4,192,a,and
...,...,...,...
1020380,24,zviad,gamsakhurdia
1020381,25,zweimal,leben
1020382,24,zwick,and
1020383,24,zydeco,music


In [3]:
# Convert to dictionary of (word1, word2): count
BIGRAM_COUNTER = bigram_data.set_index([1, 2]).to_dict()[0]
BIGRAM_COUNTER = defaultdict(int, BIGRAM_COUNTER)

BIGRAM_COUNTER_TOTAL = sum(BIGRAM_COUNTER.values())

# Preview
for item in list(BIGRAM_COUNTER.items())[:10]:
    print(item[0], item[1])

('a', 'a') 275
('a', 'aaa') 31
('a', 'all') 29
('a', 'an') 45
('a', 'and') 192
('a', 'another') 39
('a', 'at') 25
('a', 'b') 82
('a', 'b+') 45
('a', 'b-17') 26


In [4]:
# Convert to dictionary of word: count
VOCAB = defaultdict(int)

for word1, word2 in BIGRAM_COUNTER.keys():
    VOCAB[word1] += 1
    VOCAB[word2] += 1

VOCAB_TOTAL = sum(VOCAB.values())

# Preview
for item in list(VOCAB.items())[:10]:
    print(f"{item[0]}: {item[1]}")

a: 23832
aaa: 12
all: 4034
an: 5740
and: 52475
another: 1857
at: 10127
b: 167
b+: 1
b-17: 2


In [5]:
def known_words(words: list[str] | set[str]) -> set[str]:
    """Filters out words that are not in the vocabulary."""
    return set(w for w in words if VOCAB[w] > 0)


def get_word_prob(word: str) -> float:
    """Returns the probability of `word` in the corpus."""
    return VOCAB[word] / VOCAB_TOTAL


def get_bigram_prob(bigram: tuple[str, str]) -> float:
    """Returns the probability of `bigram` in the corpus."""
    return BIGRAM_COUNTER[bigram] / BIGRAM_COUNTER_TOTAL

In [6]:
from functools import lru_cache

In [7]:
@lru_cache(maxsize=None)
def get_words_at_edit_distance_n(word: str, n: int) -> set[str]:
    """Returns the set of all words at edit distance `n` from `word`."""
    if n <= 0:
        return {word}

    if n == 1:
        letters    = "abcdefghijklmnopqrstuvwxyz"
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)
    else:
        out = set()

        for word_ in get_words_at_edit_distance_n(word, n - 1):
            out |= get_words_at_edit_distance_n(word_, 1)

        return out


def get_word_candidates(word: str, edit_distance: int = 1) -> set[str]:
    """Returns candidates for a word at a given edit distance."""
    candidates = known_words([word])

    n = 1
    while not candidates and n <= edit_distance:
        candidates |= known_words(get_words_at_edit_distance_n(word, n))
        n += 1

    if not candidates:
        return {word}

    return candidates

In [8]:
def get_unigram_suggestions(
    word: str, edit_distance: int = 1
) -> list[tuple[str, float]]:
    """Returns a list of suggested words based on unigram probabilities."""
    word_candidates = get_word_candidates(word, edit_distance)

    suggestions = []
    for word in word_candidates:
        prob = get_word_prob(word)
        if prob > 0:
            suggestions.append((word, prob))

    return sorted(suggestions, key=lambda x: x[1], reverse=True)


def get_bigram_suggestions(
    bigram: tuple[str, str], edit_distance: int = 1, left=True
) -> list[tuple[tuple[str, str], float]]:
    """Returns a list of suggested bigrams based on bigram probabilities."""
    word_candidates = get_word_candidates(bigram[0 if left else 1], edit_distance)

    candidate_bigrams = set()
    for word in word_candidates:
        if left:
            candidate_bigrams.add((word, bigram[1]))
        else:
            candidate_bigrams.add((bigram[0], word))

    suggestions = []
    for c in candidate_bigrams:
        prob = get_bigram_prob(c)
        if prob > 0:
            suggestions.append((c, prob))

    return sorted(suggestions, key=lambda x: x[1], reverse=True)

In [9]:
def autocorrect(
    sentence: str,
    edit_distance: int = 1,
    vocab_word_min_prob: float = 1e-6,
    non_vocab_word_min_prob: float = 1e-15,
) -> str:
    """Returns the corrected sentence based on bigram probabilities."""
    words = sentence.split()

    n = len(words)
    if n == 0:
        return []

    sentence_corrected = []

    for i in range(n):
        if i == 0:
            right_suggestions = []
        else:
            right_suggestions = get_bigram_suggestions(
                (words[i - 1], words[i]), edit_distance, left=False
            )
            # Convert ((a, b), p) to (a, p)
            right_suggestions = [(w, p) for ((_, w), p) in right_suggestions]

        if i == n - 1:
            left_suggestions = []
        else:
            left_suggestions = get_bigram_suggestions(
                (words[i], words[i + 1]), edit_distance, left=True
            )
            # Convert ((a, b), p) to (b, p)
            left_suggestions = [(w, p) for ((w, _), p) in left_suggestions]

        suggestions = left_suggestions + right_suggestions

        # No bigram suggestions, try unigrams if the word is not in the vocabulary
        known_word = known_words([words[i]])

        if len(suggestions) == 0 and not known_word:
            suggestions = get_unigram_suggestions(words[i], edit_distance)

        # No suggestions, keep the word
        if len(suggestions) == 0:
            sentence_corrected.append(words[i])
            continue

        # Pick the word by the max sum of unigram and bigram probabilities
        probs = defaultdict(lambda: (0, 0))
        for w, p in suggestions:
            prob, count = probs[w]
            probs[w] = (prob + p, count + 1)

        # Average the probabilities
        for w, (p, c) in probs.items():
            probs[w] = p / c

        # Filter out very low probability words
        probs = {
            w: p
            for w, p in probs.items()
            if p > (vocab_word_min_prob if known_word else non_vocab_word_min_prob)
        }

        # Add max probability word to the corrected sentence
        if len(probs) == 0:
            sentence_corrected.append(words[i])
        else:
            sentence_corrected.append(max(probs, key=probs.get))

    return " ".join(sentence_corrected)

### NORGIG'S SOLUTION

In [10]:
def norvig_correction(word: str, edit_distance: int = 1) -> str:
    candidates = get_word_candidates(word, edit_distance)
    return max(candidates, key=get_word_prob)


def norvig_autocorrect(sentence: str, edit_distance: int = 1) -> str:
    sentence_corrected = []

    for word in sentence.split():
        sentence_corrected.append(norvig_correction(word, edit_distance))

    return " ".join(sentence_corrected)

In [11]:
print(norvig_autocorrect("speling korrectly"))

spelling correctly


In [12]:
# EXAMPLES (by Polina Zelenskaya)
examples = [
    "i like cokking crystal mth",
    "i lke solvng mth prblems",
    "dking sport",
    "dking patient",
]

for example in examples:
    print(f"Original:{'':>10}{example}")
    print(f"Corrected:{'':>9}{autocorrect(example)}")
    print(f"Norvig Corrected:{'':>2}{norvig_autocorrect(example)}")
    print()

Original:          i like cokking crystal mth
Corrected:         i like cooking crystal meth
Norvig Corrected:  i like cooking crystal math

Original:          i lke solvng mth prblems
Corrected:         i like solving math problems
Norvig Corrected:  i like solving math problems

Original:          dking sport
Corrected:         doing sport
Norvig Corrected:  doing sport

Original:          dking patient
Corrected:         dying patient
Norvig Corrected:  doing patient



## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

**Which ngram dataset to use.**

I chose to use the provided bigram data (bigram_data.txt) as it is a reasonably big dataset containing over one million bigram counts from a English corpus. Using trigrams or higher n-grams would provide more context but would also significantly increase the search space.

**Word probability calculation.**

For unigram word probabilities, I simply used the maximum likelihood estimate
$$P(word) = \frac{count(word)}{total\_words}.$$

**Bigram probability calculation.**
Similarly, for bigram probabilities $P(word2 | word1)$, I used the maximum likelihood estimate
$$\frac{P(word1, word2)}{sum\_w(P(word1, w))}.$$

**Edit distance candidates.**

To generate candidate words for a given word, using Norvig's solution as a basis, I first check if the word itself is in the vocabulary. If not, I generate words within increasing edit distances (1, 2, 3, ...) from the word until at least one candidate is found that exists in the vocabulary.

**Combining unigram and bigram suggestions.**

For a given word in a sentence, I generate left and right bigrams, changing only the central word. The bigram probabilities are combined in a sum to rank the suggestions. If no suggestions made using bigrams, unigrams suggestions are used instead. Very low probability suggestions are filtered out using tunable minimum probability thresholds.

**No beam search.**

For this assignment, I opted for a more straightforward approach that demonstrates the core concepts.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [13]:
class ConfusionMatrix:
    def __init__(self, TP: int = 0, FP: int = 0, TN: int = 0, FN: int = 0):
        self.TP = TP
        self.FP = FP
        self.TN = TN
        self.FN = FN

    def accuracy(self) -> float:
        num = self.TP + self.TN
        if num == 0:
            return 0

        return num / (self.TP + self.FP + self.TN + self.FN)

    def precision(self) -> float:
        if self.TP == 0:
            return 0

        return self.TP / (self.TP + self.FP)

    def recall(self) -> float:
        if self.TP == 0:
            return 0

        return self.TP / (self.TP + self.FN)

    def f1(self) -> float:
        p = self.precision()
        r = self.recall()

        if p + r == 0:
            return 0

        return 2 * p * r / (p + r)

    def __add__(self, other: "ConfusionMatrix") -> "ConfusionMatrix":
        return ConfusionMatrix(
            self.TP + other.TP,
            self.FP + other.FP,
            self.TN + other.TN,
            self.FN + other.FN,
        )

    def __repr__(self) -> str:
        return f"""\
        | {'':->10} | {'':->10} |
        | {'TP':>10} | {self.TP:>10} |
        | {'FP':>10} | {self.FP:>10} |
        | {'TN':>10} | {self.TN:>10} |
        | {'FN':>10} | {self.FN:>10} |
        | {'':->10} | {'':->10} |
        | {'Accuracy':>10} | {self.accuracy():>10.2f} |
        | {'Precision':>10} | {self.precision():>10.2f} |
        | {'Recall':>10} | {self.recall():>10.2f} |
        | {'F1':>10} | {self.f1():>10.2f} |
        | {'':->10} | {'':->10} |
        """

    def __str__(self) -> str:
        return self.__repr__()

In [14]:
def get_correction_scores(
    text_original: str, text_mutated: str, text_corrected: str
) -> ConfusionMatrix:
    words_original = text_original.split()
    words_mutated = text_mutated.split()
    words_corrected = text_corrected.split()

    confusion_matrix = ConfusionMatrix()

    for i in range(len(words_original)):
        if words_original[i] == words_mutated[i]:  # NO ERROR
            if words_original[i] == words_corrected[i]:
                confusion_matrix.TN += 1  # CORRECTLY IGNORED
            else:
                confusion_matrix.FP += 1  # INCORRECTLY CHANGED
        else:  # THERE IS AN ERROR
            if words_original[i] == words_corrected[i]:
                confusion_matrix.TP += 1  # CORRECTLY FIXED
            else:
                confusion_matrix.FN += 1  # INCORRECTLY IGNORED

    return confusion_matrix

In [15]:
def mutate_random_words_in_text(text: str, num: int, edit_distance: int = 1) -> str:
    """Randomly changes `num` words in the `text` to words with edit distance `edit_distance`."""
    words = text.split()
    if not words:
        return text

    n = len(words)
    for i in random.sample(range(n), min(n, num)):
        variants = get_words_at_edit_distance_n(words[i], edit_distance)
        variants = [v for v in variants if v]

        if variants:
            words[i] = random.choice(variants)

    return " ".join(words)

In [16]:
def preprocess_text(text: str) -> str:
    """Preprocesses the text for autocorrection."""
    # Remove all non-alphabetic characters and convert to lowercase
    text = "".join([c for c in text if c.isalpha() or c.isspace()])
    return text.lower()

In [17]:
# Loading the data
data_test = pd.read_csv("wiki_sentences_v2.csv", sep="\t")
data_test = data_test["sentence"].apply(preprocess_text)
data_test

0       confused and frustrated connie decides to leav...
1          later a womans scream is heard in the distance
2                 christian is then paralyzed by an elder
3                               the temple is set on fire
4                         outside the cult wails with him
                              ...                        
4313    confidencial also responded negatively calling...
4314    and le parisien gave the film their highest fi...
4315    the museum collection includes  film titles  p...
4316    its predecessor was the dutch historical film ...
4317            sfilmstar greta garbo by alexander binder
Name: sentence, Length: 4318, dtype: object

In [18]:
def corrector_eval(
    corrector: callable,
    data: pd.Series,
    mutated_data: pd.Series,
    edit_distance: int = 1,
) -> ConfusionMatrix:
    confusion_matrix = ConfusionMatrix()

    for sentence, sentence_mutated in tqdm(zip(data, mutated_data), total=len(data)):
        sentence_corrected = corrector(sentence_mutated, edit_distance)

        confusion_matrix += get_correction_scores(
            sentence, sentence_mutated, sentence_corrected
        )

    return confusion_matrix

In [19]:
# Evaluate the models
N = 1000
MUTATTION_RATE = 0.6  # 60% of words are mutated

sample_data = data_test.sample(N)
mutated_data = sample_data.apply(
    lambda x: mutate_random_words_in_text(x, int(MUTATTION_RATE * len(x.split())), 1)
)

confusion_matrix = corrector_eval(
  autocorrect, sample_data, mutated_data, 1
)

confusion_matrix_norvig = corrector_eval(
    norvig_autocorrect, sample_data, mutated_data, 1
)

100%|██████████| 1000/1000 [00:00<00:00, 1292.86it/s]


In [20]:
print("My Solution")
print(confusion_matrix)

My Solution
        | ---------- | ---------- |
        |         TP |       4003 |
        |         FP |        149 |
        |         TN |       4161 |
        |         FN |       1408 |
        | ---------- | ---------- |
        |   Accuracy |       0.84 |
        |  Precision |       0.96 |
        |     Recall |       0.74 |
        |         F1 |       0.84 |
        | ---------- | ---------- |
        


In [21]:
print("Norvig's Solution")
print(confusion_matrix_norvig)

Norvig's Solution
        | ---------- | ---------- |
        |         TP |       3945 |
        |         FP |        149 |
        |         TN |       4161 |
        |         FN |       1466 |
        | ---------- | ---------- |
        |   Accuracy |       0.83 |
        |  Precision |       0.96 |
        |     Recall |       0.73 |
        |         F1 |       0.83 |
        | ---------- | ---------- |
        


In conclusion, my implementation of context-sensitive spelling correction using a combination of unigram and bigram probabilities has shown marginal improvements over Norvig's.

Particularly in terms of recall and F1-score. However, both solutions achieve high accuracy, indicating their practical utility in spelling correction tasks.